<a href="https://colab.research.google.com/github/A-S-8/multi-class-sentiment-analysis/blob/main/sentnov%20dataset%20%2B%20CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install necessary libraries
!pip install transformers
!pip install tensorflow




Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 45.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 45.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 8.3 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
# Import necessary libraries
import os
import pandas as pd
import numpy as np
import re
import string
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix, classification_report
from transformers import AutoTokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Input, Embedding, Conv1D, MaxPooling1D, Flatten, Dense
from tensorflow.keras.models import Model

In [3]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [11]:
# Import necessary libraries
import pandas as pd
import numpy as np
import re
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Input, Embedding, Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

In [12]:
train = pd.read_csv('/content/drive/My Drive/sentnov/Train.csv')
test = pd.read_csv('/content/drive/My Drive/sentnov/Test.csv')
val = pd.read_csv('/content/drive/My Drive/sentnov/Val.csv')

In [13]:
def clean_text(text):
  text = text.lower()
  text = re.sub('[^a-zA-Z0-9]', ' ', text)
  text = text.split()
  text = [word for word in text if not word in stopwords]
  text = ' '.join(text)
  return text

stopwords = []
with open('/content/drive/MyDrive/sentnov/stopwords-bn.txt', 'r', encoding='utf-8') as f:
    stopwords = [line.strip() for line in f]

train['Data'] = train['Data'].apply(clean_text)
test['Data'] = test['Data'].apply(clean_text)
val['Data'] = val['Data'].apply(clean_text)

In [16]:
tokenizer = Tokenizer(num_words=100000, oov_token='<OOV>')
tokenizer.fit_on_texts(train['Data'])

train_sequences = tokenizer.texts_to_sequences(train['Data'])
test_sequences = tokenizer.texts_to_sequences(test['Data'])
val_sequences = tokenizer.texts_to_sequences(val['Data'])

train_sequences_padded = pad_sequences(train_sequences, maxlen=128, padding='post', truncating='post')
test_sequences_padded = pad_sequences(test_sequences, maxlen=128, padding='post', truncating='post')
val_sequences_padded = pad_sequences(val_sequences, maxlen=128, padding='post', truncating='post')

In [18]:
print(val.columns)

Index(['Data', 'Label'], dtype='object')


In [ ]:
# Define the CNN model
inputs = Input(shape=(128,))
x = Embedding(100000, 128, input_length=128)(inputs)
x = Conv1D(64, 5, activation='relu')(x)
x = MaxPooling1D(pool_size=4)(x)
x = Flatten()(x)
x = Dense(64, activation='relu')(x)
x = Dropout(0.5)(x)
outputs = Dense(3, activation='softmax')(x)

model = Model(inputs=inputs, outputs=outputs)

# Compile the model and train it
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.001), metrics=['accuracy'])

train_labels = pd.get_dummies(train['Label']).values
test_labels = pd.get_dummies(test['Label']).values
val_labels = pd.get_dummies(val['Label']).values

history = model.fit(train_sequences_padded, train_labels, epochs=10, batch_size=64, validation_data=(val_sequences_padded, val_labels))

# Evaluate the model on the test set and print the accuracy and confusion matrix
y_pred = np.argmax(model.predict(test_sequences_padded), axis=-1)
y_true = np.argmax(test_labels, axis=-1)